In [85]:
import numpy as np
from numpy.polynomial import Polynomial
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

from copy import deepcopy

In [114]:
train_features = pd.read_csv('../Task2/train_features.csv')
ground_truth = pd.read_csv('../Task2/train_labels.csv')
test_features = pd.read_csv('../Task2/test_features.csv')

In [115]:
pids = train_features["pid"].unique().astype(np.int)
print('number of validation patients:{}'.format(pids.size))
print('number of prediction patients:{}'.format(test_features.pid.unique().size))

number of validation patients:18995
number of prediction patients:12664


Pre Processing

In [116]:
training_mean = {}
training_std = {}
training_med = {}
for key in train_features:
    #print(key)
    training_mean[key]=np.nanmean(train_features[key])
    training_std[key]=np.nanstd(train_features[key])
    training_med[key]=np.nanmedian(train_features[key])

In [117]:
train_features

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,...,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,4,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,5,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,6,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,7,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,9999,8,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN,110.0,NaN
227936,9999,9,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,123.0,NaN
227937,9999,10,85.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,98.0,NaN,NaN,NaN,80.0,NaN,NaN,138.0,NaN
227938,9999,11,85.0,NaN,NaN,NaN,NaN,NaN,10.2,NaN,...,NaN,98.0,NaN,NaN,31.0,75.0,NaN,NaN,125.0,NaN


In [118]:
#Fill in the missing data using an imputer here i just use the median data
imputer = SimpleImputer(strategy='median').fit(train_features)
imputed_data= imputer.transform(train_features)

In [119]:
# %%time
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imputer = IterativeImputer(random_state=0).fit(train_features)
# imputed_iterative = imputer.transform(train_features)

Subtask 1

In [120]:
labels1 = [
    "LABEL_BaseExcess",
    "LABEL_Fibrinogen",
    "LABEL_AST",
    "LABEL_Alkalinephos",
    "LABEL_Bilirubin_total",
    "LABEL_Lactate",
    "LABEL_TroponinI",
    "LABEL_SaO2",
    "LABEL_Bilirubin_direct",
    "LABEL_EtCO2"
]
ytrain = ground_truth[labels1].to_numpy()

In [161]:
print(ground_truth.shape)
print(imputed_data.shape)
num_ground_truth = ground_truth.shape[0]
num_features = imputed_data.shape[1] - 2
num_hours = 12
imputed_data_slope_intersect = np.zeros((num_ground_truth, num_features*3+1))
imputed_data_slope_intersect.shape

(18995, 16)
(227940, 37)


(18995, 106)

In [168]:
current_hour

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [166]:
%%time
for i in range(num_ground_truth):
  current_hour = imputed_data[num_hours*i:num_hours*(i+1), 1]
  for j in range(num_features):
    intersect, slope = Polynomial.fit(current_hour, imputed_data[num_hours*i:num_hours*(i+1), j+2], deg=1).coef
    imputed_data_slope_intersect[i, 0]       = current_hour[-1] # The total time of stay
    imputed_data_slope_intersect[i, 3*j+1]   = np.max(imputed_data[num_hours*i:num_hours*(i+1), j+2]) #intersect
    imputed_data_slope_intersect[i, 3*j+2]   = slope
    imputed_data_slope_intersect[i, 3*j+3]   = np.var(imputed_data[num_hours*i:num_hours*(i+1), j+2])

14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
17.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
14.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


...

12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
16.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0


KeyboardInterrupt: 

In [126]:
%%time
#clf = svm.SVR(kernel='linear')
model_task1 = Ridge(alpha=1e0, fit_intercept=True)

mean_score = []
for label in labels1:
  scores = cross_val_score(model_task1, imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores)
  mean_score.append(np.mean(scores))
print(np.mean(mean_score))

[0.87690145 0.87376319 0.86923956 0.87362845 0.87776692]
[0.77130329 0.77517541 0.75308637 0.750985   0.7845481 ]
[0.70804027 0.71914376 0.69072633 0.70567522 0.70559358]
[0.711371   0.72412556 0.692805   0.7090757  0.70818751]
[0.70811414 0.71920281 0.69801468 0.70279488 0.70666273]
[0.76923962 0.77873886 0.75938378 0.76549589 0.74902116]
[0.81805291 0.80469867 0.79263754 0.82086365 0.79331542]
[0.81351077 0.80213855 0.78854654 0.7822608  0.80918147]
[0.7621977  0.75160809 0.67548207 0.69205899 0.76670807]
[0.86629335 0.87728869 0.83485634 0.83806794 0.85415649]
0.7716346854091994
Wall time: 2.18 s


In [127]:
%%time 
from sklearn.ensemble import GradientBoostingRegressor
model_task1_gradient = GradientBoostingRegressor()
for label in labels1:
  scores = cross_val_score(model_task1_gradient,imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores )

[0.92513793 0.92041918 0.92298534 0.92480689 0.92904795]
[0.79398773 0.79240298 0.81283718 0.79167062 0.80238168]
[0.7453015  0.74464544 0.73980115 0.73038982 0.73626653]
[0.75082507 0.74503248 0.74022719 0.7345673  0.74364852]
[0.74424293 0.74120773 0.74522868 0.72963626 0.74124603]
[0.80842324 0.81086912 0.80767554 0.8062764  0.79415728]
[0.8928864  0.89736512 0.89098211 0.89664052 0.89110864]
[0.8366709  0.82600696 0.82691291 0.82051028 0.83299499]
[0.7915771  0.7815053  0.74018346 0.73344365 0.78561218]
[0.9371086  0.92524544 0.92488585 0.93850884 0.93426688]
Wall time: 12min 10s


In [128]:
generated_models_gradient = []

for label in labels1:
  #model_task1 = Ridge(alpha=1e0, fit_intercept=True)
  generated_models_gradient.append(deepcopy(model_task1_gradient.fit(imputed_data_slope_intersect, ground_truth[label])))

In [129]:
labels2 = ["LABEL_Sepsis"]

In [70]:
%%time
#clf = svm.SVR(kernel='linear')
model_task2 = Ridge(alpha=1e1, fit_intercept=False)
for label in labels2:
  scores = cross_val_score(model_task2, imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores)
  print(np.mean(scores))

[0.64597036 0.67298149 0.67509715 0.68475005 0.65699689]
0.6671591885203785
Wall time: 204 ms


In [130]:
%%time 
model_task2_gradient = GradientBoostingRegressor()
for label in labels2:
  scores = cross_val_score(model_task2_gradient,imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='roc_auc')
  print(scores )

[0.69384181 0.7183145  0.73010647 0.73902746 0.69594987]
Wall time: 1min 13s


In [131]:
for label in labels2:
  #model_task2 = Ridge(alpha=1e-3, fit_intercept=False)
  generated_models_gradient.append(deepcopy(model_task2_gradient.fit(imputed_data_slope_intersect, ground_truth[label])))

Subtask 3

In [132]:
%%time
imputed_data_slope = np.zeros((num_ground_truth, 2*num_features+1))

for i in range(num_ground_truth):
  current_hour = imputed_data[num_hours*i:num_hours*(i+1), 1]
  for j in range(num_features):
    #intersect, slope = Polynomial.fit(current_hour, imputed_data[num_hours*i:num_hours*(i+1), j+2], deg=1).coef
    imputed_data_slope[i, 0]   = current_hour[-1] # The total time of stay
    imputed_data_slope[i, 2*j+1] = imputed_data_slope_intersect[i, 3*j+2]
    imputed_data_slope[i, 2*j+2] = imputed_data_slope_intersect[i, 3*j+1]
    

Wall time: 1.09 s


In [133]:
labels3 = ["LABEL_RRate", "LABEL_ABPm", "LABEL_SpO2", "LABEL_Heartrate"]

In [75]:
%%time
#clf = svm.SVR(kernel='linear')
model_task3 = [] 
alpha_list = [5e2, 5e4, 5e4, 1e4]
for i, label in enumerate(labels3):
  model_task3.append(Ridge(alpha=alpha_list[i], fit_intercept=False))
  scores = cross_val_score(model_task3[i], imputed_data_slope, ground_truth[label], cv=5, scoring='r2')
  print(scores)

[0.34500077 0.33931972 0.31279815 0.30895724 0.28331212]
[0.4826062  0.48780003 0.48723347 0.34398841 0.47137183]
[0.1817411  0.12713835 0.13672613 0.03586492 0.12947565]
[0.53786418 0.52437687 0.51215264 0.47961946 0.53645253]
Wall time: 498 ms


In [134]:
%%time 
# from sklearn.ensemble import RandomForestRegressor
model_task3_gradient = GradientBoostingRegressor()
for label in labels3:
  scores = cross_val_score(model_task3_gradient,imputed_data_slope_intersect, ground_truth[label], cv=5, scoring='r2')
  print(scores )

[0.38996123 0.36985953 0.35810399 0.36981317 0.37019512]
[0.57611868 0.5920337  0.59301779 0.55660559 0.57017734]
[0.29615129 0.21514775 0.25837716 0.35341834 0.2803078 ]
[0.61934845 0.61340083 0.59917529 0.60149172 0.62123615]
Wall time: 4min 52s


In [135]:
for  label in (labels3):
  #model_task3 = Ridge(alpha=5e4, fit_intercept=False)
  generated_models_gradient.append(deepcopy(model_task3_gradient.fit(imputed_data_slope, ground_truth[label])))

In [136]:
print(test_features.shape)
num_test_features = test_features.shape[0]
num_test_prediction = int(num_test_features/num_hours)

imputed_test_features = imputer.transform(test_features)
imputed_test_features_slope_intersect = np.zeros((num_test_prediction, num_features*3+1))
imputed_test_features_slope = np.zeros((num_test_prediction, num_features*2+1))
test_features_id = np.zeros(num_test_prediction)

(151968, 37)


In [137]:
%%time
for i in range(num_test_prediction):
  current_hour = imputed_test_features[num_hours*i:num_hours*(i+1), 1]
  test_features_id[i] = imputed_test_features[num_hours*i, 0]
  for j in range(num_features):
    intersect, slope = Polynomial.fit(current_hour, imputed_test_features[num_hours*i:num_hours*(i+1), j+2], deg=1).coef
    imputed_test_features_slope_intersect[i, 0]     = current_hour[-1] # The total time of stay
    imputed_test_features_slope_intersect[i, 3*j+1] = np.max(imputed_test_features[num_hours*i:num_hours*(i+1), j+2])
    imputed_test_features_slope_intersect[i, 3*j+2] = slope
    imputed_test_features_slope_intersect[i, 3*j+3] = np.var(imputed_test_features[num_hours*i:num_hours*(i+1), j+2])

Wall time: 1min 42s


In [138]:
%%time
for i in range(num_test_prediction):
  current_hour = imputed_test_features[num_hours*i:num_hours*(i+1), 1]
  test_features_id[i] = imputed_test_features[num_hours*i, 0]
  for j in range(num_features):
    imputed_test_features_slope[i, 0]     = current_hour[-1] # The total time of stay
    imputed_test_features_slope[i, 2*j+1] = imputed_test_features_slope_intersect[i, 3*j+2]
    imputed_test_features_slope[i, 2*j+2] = imputed_test_features_slope_intersect[i, 3*j+1]

Wall time: 748 ms


In [139]:
len(generated_models_gradient)

15

In [140]:
prediction = [test_features_id]
for model in generated_models_gradient[:-4]:
  prediction.append(model.predict(imputed_test_features_slope_intersect))
for model in generated_models_gradient[-4:]:
  prediction.append(model.predict(imputed_test_features_slope))

In [141]:
prediction = np.array(prediction)
prediction.shape

(16, 12664)

In [142]:
pd.DataFrame(prediction.T, columns=['pid'] + labels1 + labels2 + labels3).to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')

In [32]:
pd.DataFrame(prediction.T, columns=['pid'] + labels1 + labels2 + labels3).to_csv('prediction.csv', index=False, float_format='%.3f')

In [33]:
prediction.T[100]

array([1.02410000e+04, 5.56589319e-01, 1.16444016e-01, 1.53649630e-01,
       1.33200254e-01, 1.59921846e-01, 1.87440123e-01, 5.78251750e-01,
       2.01595093e-01, 1.46226099e-02, 5.53485653e-03, 6.20548748e-02,
       1.68300000e+01, 8.54880000e+01, 9.82270000e+01, 7.54270000e+01])